# Задача 3

### Часть 1
Нужно написать скрипт, который скачивает все данные прошедших президентских выборов для всех избирательных участков. 

Входная точка [по ссылке](http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227). Затем нужно перейти на сайты региональных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название региона
- название ТИК
- номер УИК
- 20 стандартных полей из итогового протокола

In [2]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

import numpy as np
import pandas as pd

In [3]:
start_link = "http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227"

headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/80.0.3987.132 YaBrowser/20.3.1.197 Yowser/2.5 Safari/537.36"
}

In [4]:
start_page = requests.get(start_link, headers=headers)
table_russia = BeautifulSoup(start_page.text)

### Получение списка полей

In [35]:
fields = ['Регион', 'ТИК', 'УИК']
for line in table_russia.body.find_all('table')[7].table.find_all('tr'):
    fields.append(line.find_all('td')[1].text)
    
del fields[3]
del fields[15]

for f in fields:
    print(f)

Регион
ТИК
УИК
Число избирателей, включенных в список избирателей 
Число избирательных бюллетеней, полученных участковой избирательной комиссией
Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно
Число избирательных бюллетеней, выданных в помещении для голосования в день голосования
Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования
Число погашенных избирательных бюллетеней
Число избирательных бюллетеней в переносных ящиках для голосования
Число бюллетеней в стационарных ящиках для голосования
Число недействительных избирательных бюллетеней
Число действительных избирательных бюллетеней
Число утраченных избирательных бюллетеней
Число избирательных бюллетеней, не учтенных при получении 
Бабурин Сергей Николаевич
Грудинин Павел Николаевич
Жириновский Владимир Вольфович
Путин Владимир Владимирович
Собчак Ксения Анатольевна
Сурайкин Максим Александрович
Титов Борис Юрьевич
Явлинский Григорий Алексеевич


In [8]:
# число округов
len(table_russia.find_all('option'))

88

###  Получение значений из таблиц

In [84]:
# table_russia = root.find_all('table')[1].find_all('tr')[1].find('td').find('form').find('select')
data_table = np.array([])

def get_table(uik_table, region_name, territory_name = "None"):
    # парсит полученную таблицу от ТИК с УИКами, считывает построчно, как есть в оригинале
    # а затем транспонирует, и таким образом верхние индексы - исходные поля протокола + необходимая доп. инфа
    
    line_tab = [0] * 24
    
    for i in range(24):
        line = []
        
        if i == 0: # нужна строка в таблице только из названия региона
            for place in root_place_table[0].find_all('td'):
                line.append(region_name)
            line_tab[i] = line
            continue
        
        if i == 1: # нужна строка в таблице только из названия ТИК
            for place in root_place_table[0].find_all('td'):
                line.append(territory_name)
            line_tab[i] = line
            continue
            
        if i == 2: # для УИК оставить только номер
            for place in root_place_table[0].find_all('td'):
                line.append(place.nobr.text.replace("УИК №",""))
            line_tab[i] = line
            continue
        
        if i == 15: # пустая строка, изначально 13, но добавляются еще две строки про регион и область
            continue
            
        for place in root_place_table[i-2].find_all('td'):
            line.append(place.nobr.text)
        line_tab[i] = line
        
    del line_tab[15] # пустая строка оставляет 0, который все ломает
    line_tab = np.array(line_tab).T
    
    global data_table
    data_table = np.append(data_table, line_tab)

### Проход по всем страницам

In [85]:
for i, region in tqdm(enumerate(table_russia.find_all('option'))):
    # 0 - поле Сумма, 43 - Калининградская область (возвращает только шапку)
    if i == 0 or i == 43:
        continue
    
    region_link = region["value"]  
    region_name = region.text
    
    region_page = requests.get(region_link, headers=headers)
    root_region = BeautifulSoup(region_page.text, "lxml")
    
    # Для города Байконур и территорий за пределми РФ
    if i == 86 or i == 87:
        link_territory_place = root_region.find_all('table')[1].find_all('td')[1].find('a')['href']
        
        place_page = requests.get(link_territory_place, headers=headers)
        root_place = BeautifulSoup(place_page.text, "lxml")
        
        root_place_table = root_place.body.find_all('table')[2].find_all('table')[7].find_all('tr')
        
        get_table(root_place_table, region_name[3:])
        continue
    
    
    # Для остальных регионов РФ
    select_region = root_region.find_all('table')[1].find_all('td')[3].find_all('table')[5].find_all('div')[0]
    
    for j, territory in enumerate(select_region.find_all('tr')[0].find_all('td')):
        
        territory_name = territory.text
        territory_link = territory.find('a')['href']
        
        territory_page = requests.get(territory_link, headers=headers)
        root_territory = BeautifulSoup(territory_page.text, "lxml")
        
        link_territory_place = root_territory.find_all('table')[1].find_all('td')[1].find('a')['href']
        place_page = requests.get(link_territory_place, headers=headers)
        root_place = BeautifulSoup(place_page.text, "lxml")
        
        # print(region_name, territory_name)
        cur = root_place
        
        try:
            root_place_table = root_place.body.find_all('table')[2].find_all('table')[7].find_all('tr')
        except: 
            if i in [3, 17, 21, 38, 48, 82]:
                # Республика Башкортостан (i = 3), Республика Татарстан (i = 17), Чеченская Республика (i = 21), 
                # Волгоградская область (i = 38), Курганская область (i = 48), Ненецкий автономный округ (i = 82)
                root_place_table = root_place.body.find_all('table')[10].find_all('div')[0].table.find_all('tr')
            elif i in [7, 12, 14, 31, 35, 46, 59, 66, 73, 77, 80]:
                # Кабардино-Балкарская Республика (i = 7), Республика Крым (i = 12), Республика Мордовия (i = 14),
                # Хабаровский край (i = 31), Белгородская область (i = 35), Кировская область (i = 46)
                # Оренбургская область (i = 59), Саратовская область (i = 66), Тульская область (i = 73)
                # Ярославская область (i = 77), город Севастополь (i = 80)
                root_place_table = root_place.body.find_all('div')[0].table.find_all('tr')
            elif i == 37:
                # Владимирская область (i = 37)
                root_place_table = root_place.body.find_all('div')[11].table.find_all('tr')
            elif i == 83: 
                # Ханты-Мансийскийй автономный округ (i = 83)
                root_place_table = root_place.body.find_all('div')[1].table.find_all('tr')
            
        get_table(root_place_table, region_name, territory_name)

88it [47:46, 32.57s/it]


### Преобразование датасета

In [86]:
# numpy все хранит как строку, необходимо изменить на нужный размер двумерного массива
data_table = np.reshape(data_table, (len(data_table) // 23, 23))

# преобразование в датафрейм (так по идее быстрее, чем когда дополнять по одной строке в цикле)
res_df = pd.DataFrame(data=data_table, columns=fields)
res_df.to_csv('vibory.csv')

# создание мультииндекса
df = res_df.set_index(['Регион', 'ТИК', 'УИК'])
df.to_csv('vibory_index.csv')
df

Число избирателей, включенных в список избирателей   \
Регион                      ТИК        УИК                                                        
 Республика Адыгея (Адыгея) Адыгейская 1                                                  2256    
                                       2                                                  2700    
                                       3                                                  2858    
                                       4                                                  2066    
                                       5                                                   700    
...                                                                                        ...    
Территория за пределами РФ  None       8433                                                436    
                                       8434                                               1141    
                                       8436                                                 78    
                                       8437                                                848    
                                       8443                                               2004    

                                            Число избирательных бюллетеней, полученных участковой избирательной комиссией  \
Регион                      ТИК        УИК                                                                                  
 Республика Адыгея (Адыгея) Адыгейская 1                                                  2181                              
                                       2                                                  2633                              
                                       3                                                  2752                              
                                       4                                                  2034                              
                                       5                                                   714                              
...                                                                                        ...                              
Территория за пределами РФ  None       8433                                               2500                              
                                       8434                                               3000                              
                                       8436                                               1507                              
                                       8437                                               2500                              
                                       8443                                               4000                              

                                            Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно  \
Регион                      ТИК        УИК                                                                                   
 Республика Адыгея (Адыгея) Адыгейская 1                                                     0                               
                                       2                                                     0                               
                                       3                                                     0                               
                                       4                                                     0                               
                                       5                                                     0                               
...                                                                                        ...                               
Территория за пределами РФ  None       8433                                                  0                            